In [5]:
path_base_r_string = r'D:\OneDrive - CELSIA S.A E.S.P'

%load_ext autoreload
%autoreload 2
import warnings
# warnings.filterwarnings("ignore")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
# this cell enables project path relative imports
import sys
from pathlib import Path
path_base = Path(path_base_r_string)
project_path = path_base / 'Proyectos'/'Eficiencia_Energetica'/'Homecenter'/'Informe_Sodimac_5001'/'informe_final'
sys.path.append(str(project_path))

In [7]:
# import all your modules here
import json
import pandas as pd
import numpy as np
# import config as cfg

from library_ubidots_v2 import Ubidots as ubi

In [12]:
# most store-level devices are correctly assigned to the homecenter organization
df_devices_org = ubi.get_available_devices_v2(label='homecenter', level='organization', page_size=1000)

# Devices belonging to the homecenter-sedes device group must be filtered out
# as they are intended for the client-side dashboard
df_devices_org = df_devices_org[~df_devices_org['device_name'].str.startswith('HC - ')]

# to get the missing devices we must request all the devices from the account
df_devices_acc = ubi.get_available_devices_v2(label=None, level='account', page_size=1000)

# not all stores are properly assigned to the organization
df_stores = df_devices_acc[df_devices_acc['device_name'].str.startswith('HC - ')]

# then filter out the missing devices
store_name_substrings = [
    'BAQ - ', # Barranquilla
    'Tintal - ', # Tintal
    'Bello - ', # Bello
    'San Juan - ', # San Juan
]

lst_df = []
for sub_str in store_name_substrings:
    is_missing = (df_devices_acc['device_name'].str.startswith(sub_str))
    lst_df.append(df_devices_acc[is_missing])

df_missing_devices = pd.concat(lst_df)

# finally we combine to get a complete set of store-level devices
df_devices = pd.concat([df_devices_org, df_missing_devices])

In [13]:
df_devices

,device_name,device_label,device_id
0,Cali Norte - Tomas Corte y Dimensionado,cali-norte---prueba-tomas-corte,62472c307ae11ca4e9c46b29
1,Cali Norte - Iluminación Patio Constructor,cali-norte---iluminacion-patio-constructor,623d292cdc4fea1a6acd1c92
2,Cali Norte - Tomas y Gondolas,cali-norte---tomas-y-gondolas,623d26f55980b1087be5e389
8,Cali Norte - Concesiones,cali-norte---concesiones,623a20ffe8f980c289d6dcf5
9,Cali Norte - ML Chiller,cali-norte---ml-chiller,623a1fb34e9390026475b0eb
...,...,...,...
456,San Juan - Aires Acondicionados,san-juan---iluminacion-principal,61ef1a7b24b83f5d398f0d2a
457,San Juan - Puente Grúa,san-juan---puente-grua,61ef1a6324b83f58b23e4a68
458,San Juan - Iluminación Parqueadero Cubierto,san-juan---iluminacion-parqueadero-cubierto,61ef1a53729df6429a9027d9
459,San Juan - Iluminación Patio Constructor,san-juan---iluminacion-patio-constructor,61ef1a16cf403e50d400cfe3


In [ ]:
df_devices_that_are_aa_ilu = pd.read_excel("ubidots_device_map.xlsx", sheet_name='AA-ILU', header=None)
df_devices_aa_ilu = df_devices[df_devices['device_name'].isin(list(df_devices_that_are_aa_ilu[0]))].copy()
df_devices_aa_ilu['type'] = df_devices_aa_ilu['device_name'].map(dict(zip(df_devices_that_are_aa_ilu[0], df_devices_that_are_aa_ilu[1])))

In [ ]:
DEVICE_IDS_TO_REQUEST = list(df_devices_aa_ilu['device_id'])
df_vars_dev = ubi.get_available_variables(DEVICE_IDS_TO_REQUEST)

df_vars = pd.merge(
    df_devices_aa_ilu,
    df_vars_dev,
    how='inner'
)


VAR_IDS_TO_REQUEST = df_vars['variable_id']
VAR_ID_TO_LABEL = dict(zip(df_vars['variable_id'], df_vars['variable_label']))

In [ ]:
CHUNK_SIZE = 10
DATE_INTERVAL_REQUEST = {'start': '2022-04-01', 'end': '2022-10-31'}

df = None
lst_responses = []
for idx in range(0, ubi.ceildiv(len(VAR_IDS_TO_REQUEST), CHUNK_SIZE)):
    idx_start = idx * CHUNK_SIZE
    idx_end = (idx + 1) * CHUNK_SIZE
    id_request_subset = VAR_IDS_TO_REQUEST[idx_start:idx_end]

    response = ubi.make_request(
        id_request_subset, 
        DATE_INTERVAL_REQUEST, 
    )

    lst_responses.append(response)

df = ubi.parse_response(lst_responses, VAR_ID_TO_LABEL)
pd.to_pickle(df, project_path / "dataframe.pkl")

In [ ]:
df

In [ ]:
set(df.loc[df['device'].str.startswith('bello'), 'device'])

In [ ]:
df_plot = df[df['device'] == 'bello---iluminacion-patio-constructor']

In [ ]:
df_plot['value'].plot()

In [ ]:
set(df.variable)

In [ ]:
energy_var_rename = {
    'ener':'energia-activa-acumulada',
    'energia-activa':'energia-activa',
    'energia-activa-acumulada':'energia-activa-acumulada',
    'energia-activa-calculada':'energia-activa'
}

In [ ]:
df = df['variable'].map()

In [ ]:
df_grouped_vars = df.groupby(['variable', 'device', 'device_name']).count().reset_index()

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
for variable in set(df.variable):
    df_sub_var = df.query('variable == @variable')

    plt.figure(figsize=(21,7))

    for device_name in set(df_sub_var['device_name']):
        df_sub_var_dev = df_sub_var.query('device_name == @device_name')

        plt.plot(df_sub_var_dev['value'], label=device_name)
    
    plt.title(variable)
    plt.legend()
    plt.show()



In [ ]:
# df_coords = Report.get_coords_dataframe(list(df_devices['id']))
# pd.to_pickle(df_coords, "data_coordinates.pkl")